In [ ]:
!pip install openai

In [ ]:
import openai
import json
from io import BytesIO
import fitz  # PyMuPDF
import pdfplumber
import PyPDF2
from pdfminer.high_level import extract_text
import pytesseract
from pdf2image import convert_from_bytes

modelo = 'gpt-3.5-turbo'

# Configura tu clave de API


In [55]:
def convert_uploaded_file_to_bytesio(uploaded_file):
    """Convierte un archivo subido en Django a un objeto BytesIO"""
    file_bytes = uploaded_file.read()  # Leer el contenido del archivo
    return BytesIO(file_bytes)  # Convertirlo en BytesIO


def extract_text_pymupdf(pdf_bytes):
    """Extraer texto por página usando PyMuPDF"""
    doc = fitz.open(stream=pdf_bytes.read(), filetype="pdf")
    return [page.get_text("text") for page in doc]


def extract_text_pdfplumber(pdf_bytes):
    """Extraer texto por página usando pdfplumber"""
    pdf_bytes.seek(0)
    with pdfplumber.open(pdf_bytes) as pdf:
        return [page.extract_text() or "" for page in pdf.pages]


def extract_text_pypdf2(pdf_bytes):
    """Extraer texto por página usando PyPDF2"""
    pdf_bytes.seek(0)
    reader = PyPDF2.PdfReader(pdf_bytes)
    return [page.extract_text() or "" for page in reader.pages]


def extract_text_pdfminer(pdf_path):
    """Extrae texto y corrige el número de páginas"""
    text = extract_text(pdf_path)
    # Separar páginas por el caracter de salto de página
    pages = text.split("\f")

    # Filtrar páginas vacías para evitar falsos positivos
    pages = [p for p in pages if p.strip()]

    return pages  # Devuelve el array de páginas


def extract_text_tesseract(pdf_bytes):
    """Extraer texto de imágenes escaneadas usando Tesseract OCR"""
    pdf_bytes.seek(0)
    images = convert_from_bytes(pdf_bytes.read())  # Convertir PDF en imágenes
    return [pytesseract.image_to_string(img) for img in images]


def extract_text_combined(pdf_file):
    """Convierte el archivo a BytesIO y almacena los textos en 5 arrays diferentes"""
    pdf_bytes = convert_uploaded_file_to_bytesio(pdf_file)

    print('generando pymupdf...')
    text_pymupdf = extract_text_pymupdf(BytesIO(pdf_bytes.getvalue()))
    print('generando pdfplumber...')
    text_pdfplumber = extract_text_pdfplumber(BytesIO(pdf_bytes.getvalue()))
    print('generando pypdf2...')
    text_pypdf2 = extract_text_pypdf2(BytesIO(pdf_bytes.getvalue()))
    print('generando pdfminer...')
    text_pdfminer = extract_text_pdfminer(BytesIO(pdf_bytes.getvalue()))
    # print('generando tesseract...')
    # text_tesseract = extract_text_tesseract(BytesIO(pdf_bytes.getvalue()))

    return text_pymupdf, text_pdfplumber, text_pypdf2, text_pdfminer  # , text_tesseract


def extraer_texto_pdf(file):
    text_pymupdf, text_pdfplumber, text_pypdf2, text_pdfminer = extract_text_combined(
        file)

    file.close()

    respuesta = ''

    for i in range(len(text_pdfminer)):
        text = '-¿!11441165473941=(-' + text_pymupdf[i] + '-¿!11441165473941=(-' + text_pdfplumber[i] + '-¿!11441165473941=(-' + \
            text_pypdf2[i] + '-¿!11441165473941=(-' + \
            text_pdfminer[i]  # + '-¿!11441165473941=(-' + text_tesseract[i]
        try:
            response = client.chat.completions.create(
                model=modelo,
                messages=[
                    {"role": "system",
                        "content": "He usado 5 librerías para extraer el texto de un pdf. Los 4 resultados son los siguientes. Necesito que me des un texto limpio final usando la información de las 4 extracciones. No escribas nada más que el texto limpio final. La separación entre texto y texto es: '-¿!11441165473941=(-'"},
                    {"role": "user", "content": text},
                ],
                temperature=0.7
            )
            respuesta += response.choices[0].message.content
        except Exception as e:
            return f"Error en la solicitud a OpenAI: {str(e)}"

    return respuesta

In [63]:
# Función para analizar el temario y extraer apartados y subapartados
def analizar_temario(temario_texto):
    prompt = """
    Eres experto en analizar temarios académicos.
    Analiza el texto proporcionado y devuelve los apartados y subapartados en formato array.

    Ejemplo del formato esperado:
    [
        "BLOQUE I. FISIOLOGÍA DE LA AUDICIÓN HUMANA",
        "TEMA 1. ESTUDIO FÍSICO DEL SONIDO",
        "1. FUENTE SONORA VIBRANTE",
        "1.1. Cualidades esenciales de las ondas sonoras",
        "2. MEDIO DE PROPAGACIÓN",
        "a) REFLEXIÓN, ECO Y REVERBERACIÓN",
        "b) REFRACCIÓN",
        "Absorción",
        "Resonancia",
        "Efecto Doppler"
    ]
    """
    respuesta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "system", "content": prompt}, {"role": "user", "content": temario_texto}]
    )

    try:
        apartados = json.loads(respuesta.choices[0].message.content)
    except json.JSONDecodeError as e:
        print(f"Error al decodificar el JSON: {e}")
    return apartados

# Función para generar preguntas tipo test en el formato JSON solicitado

def generar_preguntas_json(texto, apartados, cantidad=20):
    # prompt = f"""
    # Eres experto en crear preguntas tipo test sobre temarios académicos.
    # Te voy a entregar un texto y luego te pediré preguntas tipo test sobre él.
    # Las preguntas tipo test deben crearse con el siguiente formato JSON:
    # {{
    # "preguntas": [
    #     {{
    #     "texto": "Enunciado de la pregunta 1",
    #     "respuestas": [
    #         {{
    #         "texto": "Respuesta CORRECTA"
    #         }},
    #         {{
    #         "texto": "Respuesta 2"
    #         }},
    #         {{
    #         "texto": "Respuesta 3"
    #         }},
    #         {{
    #         "texto": "Respuesta 4"
    #         }}
    #     ],
    #     "ayuda": "Explicación de la pregunta usando las palabras literales que aparecen en el texto del temario."
    #     }},
    #     {{
    #     "texto": "Enunciado de la pregunta 2",
    #     "respuestas": [
    #         {{
    #         "texto": "Respuesta CORRECTA"
    #         }},
    #         {{
    #         "texto": "Respuesta 2"
    #         }},
    #         {{
    #         "texto": "Respuesta 3"
    #         }},
    #         {{
    #         "texto": "Respuesta 4"
    #         }}
    #     ],
    #     "ayuda": "Explicación de la pregunta usando las palabras literales que aparecen en el texto del temario."
    #     }}
    # ]
    # }}
    # Es muy importante que la respuesta correcta sea siempre la primera, en todas las preguntas debe ser así.
    
    # El texto con el temario es el siguiente:\n{texto}
    # """
    # messages = [{"role": "system", "content": prompt}]
    # for apartado in apartados:
    #     messages.append({"role": "user", "content": f"""Crea {cantidad} preguntas tipo test sobre el apartado {apartado} del texto"""})

    # respuesta = client.chat.completions.create(
    #     model=modelo,
    #     messages=messages,
    #     response_format={"type": "json_object"}
    # )

    # preguntas_json = respuesta.choices[0].message.content
    # print('--------------')
    # print('PREGUNTAS')
    # print(preguntas_json)
    # print('--------------')
    # return json.loads(preguntas_json)
    
    preguntas_completas = []
    for apartado in apartados:
        print(apartado)
        prompt = f"""
        Eres experto en crear preguntas tipo test sobre temarios académicos.
        Genera 20 preguntas tipo test sobre el apartado "{apartado}" del texto proporcionado.
        La finalidad de las preguntas es conseguir estudiar y aprender de manera exitosa el temario.
        
        Las preguntas deben estar en el siguiente formato JSON:
        {{
            "preguntas": [
                {{
                    "texto": "Enunciado de la pregunta",
                    "respuestas": [
                        {{"texto": "Respuesta CORRECTA"}},
                        {{"texto": "Respuesta 2"}},
                        {{"texto": "Respuesta 3"}},
                        {{"texto": "Respuesta 4"}}
                    ],
                    "ayuda": "Explicación de la pregunta usando las palabras literales que aparecen en el texto del temario."
                }}
            ]
        }}

        El texto del temario es el siguiente:
        {texto}
        """
        
        respuesta = client.chat.completions.create(
            model=modelo,
            messages=[{"role": "system", "content": prompt}],
            response_format={"type": "json_object"}
        )

        # Agregar las preguntas generadas a la lista completa
        try:
            preguntas_json = json.loads(respuesta.choices[0].message.content)
            print(preguntas_json, "\n\n")
            preguntas_completas.extend(preguntas_json["preguntas"])
        except json.JSONDecodeError as e:
            print(f"Error en el JSON del apartado {apartado}: {e}")

    print(f"Se generaron {len(preguntas_completas)} preguntas en total.")
    return preguntas_completas

# Función para analizar la cobertura del temario

def analizar_cobertura(temario, preguntas_generadas):
    prompt = f"Analiza si las siguientes preguntas cubren todo el temario:\nTemario: {temario}\nPreguntas: {preguntas_generadas}\nIndica los apartados no cubiertos."

    respuesta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=500
        response_format={"type": "json_object"}
    )

    temas_no_cubiertos = respuesta.choices[0].message.content
    print('--------------')
    print('no cobertura')
    print(temas_no_cubiertos)
    print('--------------')
    return temas_no_cubiertos.split("\n")

# Función para estructurar todas las preguntas en un JSON unificado

def estructurar_en_json(apartados, preguntas_por_apartado):
    resultado = {
        "preguntas": []
    }

    for apartado, preguntas in zip(apartados, preguntas_por_apartado):
        resultado["preguntas"].extend(preguntas["preguntas"])

    return json.dumps(resultado, indent=4, ensure_ascii=False)

# Función principal para generar la batería completa de preguntas

def generar_bateria_completa(temario_texto):
    # Paso 1: Análisis del temario
    apartados = analizar_temario(temario_texto)
    # print('\n\n', apartados)
    # Paso 2: Generar 20 preguntas por cada apartado en el formato JSON solicitado
    # preguntas = []
    # for apartado in apartados:
    #     preguntas.append(generar_preguntas_json(temario_texto, apartados, 20))
    # print('\n\n', preguntas)
    
    preguntas_generadas = generar_preguntas_json(temario_texto, apartados, 20)

    # # Paso 3: Analizar la cobertura del temario
    # preguntas_generadas = json.dumps(preguntas_por_apartado)
    # temas_no_cubiertos = analizar_cobertura(temario_texto, preguntas_generadas)

    # # Paso 4: Generar 10 preguntas adicionales para temas no cubiertos
    # preguntas_adicionales = [
    #     generar_preguntas_json(tema, numero_apartado, 10)
    #     for numero_apartado, tema in enumerate(temas_no_cubiertos, start=1)
    # ]

    # # Unir las preguntas originales con las adicionales
    # preguntas_por_apartado.extend(preguntas_adicionales)

    # # Paso 5: Devolver el JSON con el formato especificado
    # resultado_json = estructurar_en_json(apartados, preguntas_por_apartado)
    
    return preguntas_generadas


resultado = generar_bateria_completa(temario)
print(resultado)

BLOQUE I. FISIOLOGÍA DE LA AUDICIÓN HUMANA
{'preguntas': [{'texto': '¿Qué son las ondas sonoras?', 'respuestas': [{'texto': 'Ondas mecánicas de presión que se propagan en un medio elástico y denso'}, {'texto': 'Ondas electromagnéticas que se propagan en el aire'}, {'texto': 'Ondas de luz que se mueven en el agua'}, {'texto': 'Ondas eléctricas que se transmiten por el metal'}], 'ayuda': 'Las ondas sonoras son aquellas que, tras pasar por nuestro sistema auditivo, se convierten en información que el cerebro descodifica. Son ondas mecánicas de presión que se propagan en un medio elástico y denso.'}, {'texto': '¿Cuál es la unidad de medida de la frecuencia?', 'respuestas': [{'texto': 'Hercios (Hz)'}, {'texto': 'Decibelios (dB)'}, {'texto': 'Pascales (Pa)'}, {'texto': 'Vatios (W)'}], 'ayuda': 'La frecuencia se mide en hercios (Hz).'}, {'texto': '¿Qué cualidad del sonido está relacionada con la amplitud?', 'respuestas': [{'texto': 'Intensidad'}, {'texto': 'Timbre'}, {'texto': 'Frecuencia'}, 

In [ ]:
from django.core.files.uploadedfile import SimpleUploadedFile

ruta_al_archivo = '/home/monica/Descargas/fisio_pdf/Tema_1_fisio.pdf'
with open(ruta_al_archivo, 'rb') as f:
    contenido_archivo = f.read()

archivo_simulado = SimpleUploadedFile("archivo.txt", contenido_archivo)

temario = extraer_texto_pdf(archivo_simulado)

In [62]:
print(preguntas_generadas)

NameError: name 'preguntas_generadas' is not defined

--------------
APARTADOS
{
    "apartados": [
        {
            "titulo": "Tema 1. ESTUDIO FÍSICO DEL SONIDO",
            "subapartados": [
                {"titulo": "1. FUENTE SONORA VIBRANTE"},
                {"titulo": "1.1. Cualidades esenciales de las ondas sonoras",
                    "subapartados": [
                        {"titulo": "a) FRECUENCIA / TONO"},
                        {"titulo": "b) AMPLITUD / INTENSIDAD"}
                    ]
                }
            ]
        },
        {
            "titulo": "2. MEDIO DE PROPAGACIÓN",
            "subapartados": [
                {"titulo": "Factores que afectan a la propagación del sonido",
                    "subapartados": [
                        {"titulo": "a) REFLEXIÓN, ECO Y REVERBERACIÓN"},
                        {"titulo": "b) REFRACCIÓN"},
                        {"titulo": "Absorción"},
                        {"titulo": "Resonancia"},
                        {"titulo": "Efecto Doppler"}
          